<a href="https://colab.research.google.com/github/Ahmed-Ragab/machine-learning-zoomcamp/blob/main/lead_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [3]:
!wget $data -O Lead_Score.csv

--2025-10-13 20:36:49--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘Lead_Score.csv’

Lead_Score.csv      100%[===================>]  78.98K  --.-KB/s    in 0.008s  

2025-10-13 20:36:49 (10.0 MB/s) - ‘Lead_Score.csv’ saved [80876/80876]



In [4]:
df = pd.read_csv('Lead_Score.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


**Data preparation**

In [5]:
# Check if the missing values are presented in the features.
df.isnull().sum()

,0
lead_source,128
industry,134
number_of_courses_viewed,0
annual_income,181
employment_status,100
location,63
interaction_count,0
lead_score,0
converted,0


In [18]:
# Replace null values in categorical features with 'NA'
categorical_features = df.select_dtypes(include='object').columns
df[categorical_features] = df[categorical_features].fillna('NA')

In [32]:
# Replace null values in numerical features with 0.0
numerical_features = list(df.select_dtypes(include='number').columns)
if 'converted' in numerical_features:
    numerical_features.remove('converted')
df[numerical_features] = df[numerical_features].fillna(0.0)

**Question 1**

In [33]:
# What is the most frequent observation (mode) for the column industry?
df['industry'].groupby(df['industry']).count().sort_values(ascending=False)



,industry
industry,
retail,203
finance,200
other,198
education,187
healthcare,187
technology,179
manufacturing,174
NA,134


**Question 2**

In [34]:
# Create the correlation matrix for the numerical features of your dataset
corr_matrix = df[numerical_features].corr()
corr_matrix



,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [25]:
df[numerical_features].corrwith(df.converted).abs()


,0
number_of_courses_viewed,0.435914
annual_income,0.053131
interaction_count,0.374573
lead_score,0.193673
converted,1.000000


**Split the data**

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
# Split your data in train/val/test sets with 60%/20%/20% distribution. set the seed to 42
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [37]:
# reset index for each df
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [38]:
# Build y data (target variable) for each df "train, val, and test"
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [39]:
# Delete the target variable from the DFs
del df_train['converted']
del df_val['converted']
del df_test['converted']

**Question 3**

In [41]:
from sklearn.metrics import mutual_info_score

In [58]:
# Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.
def mutual_info_converted_score(series):
    return mutual_info_score(series, df_train_full.converted)

df_mi = df_train_full[categorical_features].apply(mutual_info_converted_score)
df_mi = round(df_mi.sort_values(ascending=False),2)
df_mi


,0
lead_source,0.03
employment_status,0.01
industry,0.01
location,0.00


**Question 4**

In [48]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [51]:
# Use Scikit-Learn to encode categorical features
dv = DictVectorizer(sparse=False)

# Combine categorical and numerical features
all_features = categorical_features.tolist() + numerical_features

train_dict = df_train[all_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[all_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [52]:
# Train a logistic regression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [56]:
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
round(model.score(X_val, y_val),2)

0.7

**Question 5. Feature selection**

In [ ]:
# find the least useful feature using the feature elimination technique


**Question 6. Parameter tuning**